# Decision tree and Random forest

#### Main concepts

**Entropy**\
It's a measure of disorder (or purity, we'll see what it means). If we are working with a dichotomous variablee, we can interpet it as follow: if $E=0$ (resp. $E=1$), all observations are in the first (resp. second) class. Otherwise, if $E=0.5$, the disorder is at the highest (the categories are equally divided). More formally, we can describe it as the following.
$$E = \sum_{i=1}^c -p_i log_2 p_i,~\in [0,1]$$
$$c = the~number~of~classes$$
Have in mind that $E \in [0,1]$ is only true when working with dichotomous variable, even tough the logic explained before is the same for variables with more than 2 categories. Let's try an example: first we create 3 different variables with different number of modalities.


In [19]:
import random

two_modalities = ['Yes'] * 7 + ['No'] * 3
random.shuffle(two_modalities)
print(f"My first variable (dichotomous): {two_modalities}")

three_modalities = ['Yes'] * 3 + ['No'] * 3 + ['Maybe'] * 3
random.shuffle(three_modalities)
print(f"My second variable: {three_modalities}")

four_modalities = ['A'] * 3 + ['B'] * 3 + ['C'] * 3 + ['D'] * 3
random.shuffle(four_modalities)
print(f"My thirds variable: {four_modalities}")

My first variable (dichotomous): ['Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes']
My second variable: ['Maybe', 'Maybe', 'No', 'No', 'Yes', 'Maybe', 'Yes', 'No', 'Yes']
My thirds variable: ['B', 'A', 'D', 'A', 'D', 'C', 'B', 'B', 'A', 'D', 'C', 'C']


In [38]:
#modules
import math
from scipy.stats import entropy
import numpy as np

#define a function
def my_entropy(data):
    classes = set(data)
    n = len(data)
    ent = 0
    pk = []
    for c in classes:
        p = data.count(c) / n
        pk.append(p)
        ent -= p * math.log(p,2)
    print(f"Entropy of the variable (computed by hand, with {len(classes)} modalities): {ent}")
    
    ent2 = entropy(np.array(pk),base=2)
    print(f"Entropy of the variable (computed with scipy, with {len(classes)} modalities): {ent2}\n")

my_entropy(two_modalities)
my_entropy(three_modalities)
my_entropy(four_modalities)

Entropy of the variable (computed by hand, with 2 modalities): 0.8812908992306927
Entropy of the variable (computed with scipy, with 2 modalities): 0.8812908992306927

Entropy of the variable (computed by hand, with 3 modalities): 1.584962500721156
Entropy of the variable (computed with scipy, with 3 modalities): 1.584962500721156

Entropy of the variable (computed by hand, with 4 modalities): 2.0
Entropy of the variable (computed with scipy, with 4 modalities): 2.0



There are other similar measure as entropy, like [Gini index](https://blog.quantinsti.com/gini-index/#:~:text=Gini%20Index%20is%20a%20powerful,of%20a%20decision%20tree%20model.). A Gini index of 0 means that the sample is perfectly homgeneous (every observation is from the same class) and an index of 1 means is the opposite. We describe it formally we the following formula:
$$G_{ini} = 1 - \sum_{i=1}^c p_i^2,~\in [0,1]$$
$$c = the~number~of~classes$$


**But what are decision trees?** \
Decision trees are a way to formally define a decision process, based on some condition (i.e. what we are looking for in ML). More concretly, you can see it that way (image taken from [Kaggle](https://www.kaggle.com/learn)):

<div style="text-align:center">
  <img src="imageDT.png" alt="Decision tree example" width="600" height="400"/>
</div>

Our goal here, for a given target variable (ex: the house price) and a dataset, find what are the nodes (ex: "Does house have more than 2 Bedrooms") that predicts the best the target variable. We will train a model on a training dataset, in order to find those nodes, and then test the model on new data in order to compare what the model predict and the true value (we will see some example after).

<br>

#### Algorithms used in decision tree

Since decision trees can be used either for regression task (predict a continuous variable, like monthly income) or classification task (predict a categorical variable, like marital status), we will have to see both of them. We will only talk about the [CART](https://fr.wikipedia.org/wiki/Algorithme_CART) (classification and regression tree) algorithm, even tough there are lots of other with different specificities. The CART algorithm uses the Gini index as a main metric to evaluate each split for classification task and least square for feature selection for regression task.


**CART algorithm**\
We will implement our own CART algorithm for a classification task. First, we create a Gini index function:

In [41]:
def my_gini_index(data):
    
    sum_of_p = 0 #init the index at 0 since it's a sum
    classes = set(data) #define a list of all different classes
    n = len(data) #save the length of data
    
    #iterate over all classes and add the square proportion of each modality to the gini index
    for c in classes:
        p = data.count(c) / n
        sum_of_p += p**2
        
    #define gini using the sum
    gini = 1 - sum_of_p
    
    #print the index value
    print(f"Gini index (with {len(classes)} modalities): {gini}")
    
    #output
    return gini

my_gini_index(two_modalities)
my_gini_index(three_modalities)
my_gini_index(four_modalities)

Gini index (with 2 modalities): 0.42000000000000004
Gini index (with 3 modalities): 0.6666666666666667
Gini index (with 4 modalities): 0.75


0.75

We have consistent results: the higher the number of modalities, the higher the Gini index. Before going further, we need to define a dataset that will be used for the continuation.

In [53]:
import numpy as np
import pandas as pd

#define a sample size
size = 12

#create 3 variables
education = np.random.choice(['low', 'medium', 'high'],
                             size=size,
                             replace=True)
income = np.random.choice(['none', 'low', 'medium', 'high', 'very high'],
                                 size=size,
                                 replace=True)
gender = np.random.choice(['men', 'women', 'other'],
                                 size=size,
                                 replace=True)
happiness = np.random.choice(['Yes', 'No'],
                                 size=size,
                                 replace=True)

#create a pandas dataframe
data = {'Education': education, 'Income': income, 'Gender': gender, 'Happiness': happiness}
df = pd.DataFrame(data)

#display the df
print(df)

   Education     Income Gender Happiness
0       high        low  women        No
1     medium       none  other       Yes
2       high        low    men        No
3       high       high  women       Yes
4     medium        low  women       Yes
5        low     medium  other       Yes
6        low  very high    men        No
7       high     medium    men        No
8       high        low  other        No
9        low       none    men       Yes
10      high     medium  women       Yes
11       low        low    men        No


We want to create a decision tree that discriminates if a person is happy or not, based on the person education level, income level and gender. Important: my data are randomly generated and purely fake (the results obtained are not replicable and cannot be used make inference for real world people). Now let's briefly explain how the algorithm works:
- (1) for each of the $k$ (= 3) explanatory variable, we create a contingency table with the target variable (Happiness). Then we compute the Gini index of each modality (low, medium and high, for Education). And we compute the Gini index (sum of  the latter) of each variable. 
- (2) We select the variable with the least Gini index (the root node) since it means the highest purity and homogeneity.
- (3) With the latter variable, we create $c$ branchs (the number of modality of the variable in question). For Education, we have a branch low, medium and high. We are now looking for the split or node to add at the end of each of these branch.
- (4) We repeat the step (1) and (2) on each new node but by doing subset of the initial training set (Education == "low") until the Gini index equals 0. Once it's the case, it's homogenous and we add a leaf with the only one modality from the contingency table (there can only be one since the Gini index equals 0, by definition). 

Let's do an example in order to make it clearer!

In [68]:
import pandas as pd

#create a contingency table between two variables
contingency_tab = pd.crosstab(df.Education, df.Happiness)

#calculate the number of modalities in the explanatory variable
number_of_modalities = len(contingency_tab)

#compute the Gini index for each modality
for i in range(number_of_modalities):
    mod = contingency_tab.iloc[i]
    print(my_gini_index())

TypeError: my_gini_index() missing 1 required positional argument: 'data'

<br>

<br>

# Implementation with Python

#### Import of the data

In [ ]:
import pandas as pd
import os
data = pd.read_csv(f"/Users/josephbarbier/Desktop/appML/models/Iris.csv")
data.head()